In [1]:
!pip install delta-spark==3.1.0

### Etapa 1: Inicialização da sessão Spark (camada Bronze)

Nesta etapa inicializo a sessão Spark configurada com suporte ao Delta Lake. Essa sessão será utilizada para ler o arquivo original da base IBM HR, realizar a primeira ingestão dos dados e gravar a tabela na camada Bronze.

**Resultado esperado:**  
A sessão Spark é criada com sucesso e exibida na saída da célula, indicando que o ambiente está pronto para iniciar a ingestão dos dados na camada Bronze.


In [2]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = (
    SparkSession.builder
        .appName("mvp_turnover")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark


### Etapa 2: Leitura do arquivo CSV original (camada Bronze)

Nesta etapa realizo a leitura do arquivo CSV original da base IBM HR (`WA_Fn-UseC_-HR-Employee-Attrition.csv`) diretamente da camada Bronze. Utilizo a opção `header=true` para considerar a primeira linha como cabeçalho e `inferSchema=true` para que o Spark tente identificar automaticamente os tipos de dados de cada coluna. Em seguida, exibo as 5 primeiras linhas e o schema completo do DataFrame.

**Resultado esperado:**  
Carregar corretamente os dados brutos do arquivo CSV no DataFrame `df_bronze` e visualizar sua estrutura inicial, que servirá de base para as próximas etapas do pipeline.


In [3]:
path_bronze = "/home/jovyan/data/bronze/WA_Fn-UseC_-HR-Employee-Attrition.csv"

df_bronze = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(path_bronze)
)

df_bronze.show(5)
df_bronze.printSchema()


+---+---------+-----------------+---------+--------------------+----------------+---------+--------------+-------------+--------------+-----------------------+------+----------+--------------+--------+--------------------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+------------------------+-------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Age|Attrition|   BusinessTravel|DailyRate|          Department|DistanceFromHome|Education|EducationField|EmployeeCount|EmployeeNumber|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|             JobRole|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|Over18|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalanc

### Etapa 3: Escrita da camada Bronze em formato Delta

Nesta etapa salvo o DataFrame `df_bronze`, carregado a partir do arquivo CSV original, em formato Delta na pasta da camada Bronze. Esse passo converte a ingestão inicial em uma tabela Delta estruturada, que será utilizada como fonte para as transformações da camada Silver.

**Resultado esperado:**  
A tabela Bronze é gravada com sucesso no diretório `/home/jovyan/data/bronze/delta_ibm_hr`, ficando disponível para leitura e processamento nas etapas seguintes do pipeline.


In [6]:
path_delta_bronze = "/home/jovyan/data/bronze/delta_ibm_hr"

df_bronze.write.format("delta").mode("overwrite").save(path_delta_bronze)


In [7]:
!ls /home/jovyan/data/bronze


delta_ibm_hr  WA_Fn-UseC_-HR-Employee-Attrition.csv


### Etapa 4: Validação da tabela Bronze em formato Delta

Nesta etapa leio a tabela Bronze já salva em formato Delta, a partir do caminho `/home/jovyan/data/bronze/delta_ibm_hr`. Em seguida, exibo as 5 primeiras linhas e o schema completo do DataFrame `df_bronze_delta` para verificar se a escrita em Delta ocorreu corretamente.

**Resultado esperado:**  
Confirmar que a tabela Bronze foi persistida com sucesso em formato Delta, mantendo a estrutura e os dados esperados para servir de base às transformações da camada Silver.


In [8]:
path_delta_bronze = "/home/jovyan/data/bronze/delta_ibm_hr"

df_bronze_delta = spark.read.format("delta").load(path_delta_bronze)

df_bronze_delta.show(5)
df_bronze_delta.printSchema()


+---+---------+-----------------+---------+--------------------+----------------+---------+--------------+-------------+--------------+-----------------------+------+----------+--------------+--------+--------------------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+------------------------+-------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Age|Attrition|   BusinessTravel|DailyRate|          Department|DistanceFromHome|Education|EducationField|EmployeeCount|EmployeeNumber|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|             JobRole|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|Over18|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalanc